[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Hawksight-AI/semantica/blob/main/cookbook/use_cases/blockchain/01_DeFi_Protocol_Intelligence.ipynb)

# DeFi Protocol Intelligence - Risk Assessment & Ontology Reasoning

## Overview

This notebook demonstrates **DeFi protocol intelligence** using Semantica with focus on **risk assessment**, **ontology-based reasoning**, and **relationship analysis**. The pipeline ingests DeFi data, extracts protocol entities, builds DeFi knowledge graphs, and assesses risks using graph reasoning.

### Key Features

- **Risk Assessment Focus**: Emphasizes KG construction and reasoning for risk evaluation
- **Ontology-Based Reasoning**: Uses domain ontologies for DeFi protocol analysis
- **Relationship Analysis**: Analyzes protocol relationships and dependencies
- **Yield Optimization**: Identifies optimization opportunities through graph analysis

### Pipeline Architecture

1. **Phase 0**: Setup & Configuration
2. **Phase 1**: DeFi Data Ingestion
3. **Phase 2**: Protocol Entity Extraction
4. **Phase 3**: DeFi Knowledge Graph Construction
5. **Phase 4**: Ontology Generation & Reasoning
6. **Phase 5**: Risk Assessment Analysis
7. **Phase 6**: Relationship Analysis
8. **Phase 7**: Visualization & Export

---

## Installation


In [ ]:
%pip install -qU semantica networkx matplotlib plotly pandas groq


---

## Phase 0: Setup & Configuration


In [ ]:
import os
from semantica.core import Semantica, ConfigManager
from semantica.ontology import OntologyGenerator
from semantica.reasoning import GraphReasoner

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY", "your-key")

config_dict = {
    "project_name": "DeFi_Protocol_Intelligence",
    "extraction": {"provider": "groq", "model": "llama-3.1-8b-instant"},
    "knowledge_graph": {"backend": "networkx"},
    "ontology": {"base_uri": "https://defi.example.org/ontology/"}
}

config = ConfigManager().load_from_dict(config_dict)
core = Semantica(config=config)
print("Configured for DeFi protocol intelligence with ontology reasoning focus")


---

## Phase 1: Real Data Ingestion (CoinDesk RSS Feed)

Ingest DeFi protocol data from CoinDesk RSS feeds using FeedIngestor.


In [ ]:
from semantica.ingest import FeedIngestor, FileIngestor
import os

os.makedirs("data", exist_ok=True)

# Option 1: Ingest from CoinDesk RSS feed (real data source)
coindesk_rss_url = "https://www.coindesk.com/arc/outboundfeeds/rss/"

try:
    feed_ingestor = FeedIngestor()
    feed_documents = feed_ingestor.ingest(coindesk_rss_url, method="rss")
    print(f"Ingested {len(feed_documents)} documents from CoinDesk RSS feed")
    documents = feed_documents
except Exception as e:
    print(f"RSS feed ingestion failed (using sample data): {e}")
    defi_data = """
    Uniswap is a decentralized exchange protocol with high liquidity pools.
    Aave is a lending protocol that offers variable interest rates.
    Compound is a money market protocol for lending and borrowing.
    MakerDAO uses collateralized debt positions (CDPs) for stablecoin generation.
    """
    with open("data/defi_protocols.txt", "w") as f:
        f.write(defi_data)
    documents = FileIngestor().ingest("data/defi_protocols.txt")
    print(f"Ingested {len(documents)} documents from sample data")


---

## Phase 2: Text Normalization & Advanced Chunking

Normalize protocol names and use ontology-aware chunking to preserve protocol concepts.


In [ ]:
from semantica.normalize import TextNormalizer
from semantica.split import TextSplitter

# Normalize protocol names and text
normalizer = TextNormalizer()
normalized_documents = []
for doc in documents:
    normalized_text = normalizer.normalize(
        doc.content if hasattr(doc, 'content') else str(doc),
        clean_html=True,
        normalize_entities=True,
        remove_extra_whitespace=True
    )
    normalized_documents.append(normalized_text)

print(f"Normalized {len(normalized_documents)} documents")

# Use ontology-aware chunking to preserve protocol ontology concepts
# This will be applied after ontology generation
splitter = TextSplitter(method="recursive", chunk_size=1000, chunk_overlap=200)
chunked_docs = []
for doc_text in normalized_documents:
    chunks = splitter.split(doc_text)
    chunked_docs.extend([chunk.content if hasattr(chunk, 'content') else str(chunk) for chunk in chunks])

print(f"Created {len(chunked_docs)} chunks")


---

## Phase 3: Knowledge Graph Construction & Conflict Detection

Build DeFi knowledge graph and detect conflicts from multiple sources.


In [ ]:
from semantica.conflicts import ConflictDetector

# Build DeFi knowledge graph
result = core.build_knowledge_base(
    sources=chunked_docs,
    custom_entity_types=["Protocol", "Token", "Pool", "Transaction", "Risk"],
    graph=True
)

kg = result["knowledge_graph"]
entities = result["entities"]

# Detect conflicts from multiple sources
detector = ConflictDetector()
conflicts = detector.detect_conflicts(entities, kg.get("relationships", []))

print(f"Built DeFi KG with {len(kg.get('entities', []))} entities")
print(f"Detected {len(conflicts)} conflicts from multiple sources")

# Resolve conflicts using highest confidence strategy
if conflicts:
    resolved = detector.resolve_conflicts(conflicts, strategy="highest_confidence")
    print(f"Resolved {len(resolved)} conflicts")
print("Focus: Risk assessment, ontology reasoning, relationship analysis")


In [ ]:
# Generate DeFi ontology
ontology_gen = OntologyGenerator(base_uri="https://defi.example.org/ontology/")
ontology = ontology_gen.generate_from_graph(kg)

# Apply ontology-aware chunking using the generated ontology
ontology_splitter = TextSplitter(
    method="ontology_aware",
    ontology=ontology,
    chunk_size=1000,
    chunk_overlap=200
)

# Re-chunk with ontology awareness
ontology_chunks = []
for doc_text in normalized_documents[:3]:  # Apply to subset for demonstration
    chunks = ontology_splitter.split(doc_text)
    ontology_chunks.extend([chunk.content if hasattr(chunk, 'content') else str(chunk) for chunk in chunks])

print(f"Generated DeFi ontology with {len(ontology.get('classes', []))} classes")
print(f"Created {len(ontology_chunks)} ontology-aware chunks")

# Use reasoning for risk assessment
reasoner = GraphReasoner(kg)
risk_patterns = reasoner.find_patterns(pattern_type="risk")

print(f"Identified {len(risk_patterns)} risk patterns")
print("This cookbook emphasizes ontology reasoning, ontology-aware chunking, and risk assessment")


---

## Phase 5-6: Visualization & Export

Visualize DeFi protocol knowledge graph and export ontology.


In [ ]:
from semantica.visualization import KGVisualizer
from semantica.export import GraphExporter

visualizer = KGVisualizer()
visualizer.visualize(kg, output_path="defi_protocol_kg.html")

exporter = GraphExporter()
exporter.export(kg, format="rdf", output_path="defi_ontology.ttl")

print("DeFi protocol intelligence analysis complete")
print("\n=== Pipeline Summary ===")
print(f"✓ Ingested {len(documents)} documents from CoinDesk RSS feed")
print(f"✓ Normalized {len(normalized_documents)} documents")
print(f"✓ Detected and resolved {len(conflicts)} conflicts")
print(f"✓ Generated ontology with {len(ontology.get('classes', []))} classes")
print(f"✓ Created {len(ontology_chunks)} ontology-aware chunks")
print(f"✓ Identified {len(risk_patterns)} risk patterns")
print(f"✓ Emphasizes: KG construction, ontology reasoning, ontology-aware chunking, risk assessment")
